In [1]:
# %%
# %%
from datasets import load_dataset
import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
import pdb
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from utils import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
import logging
import sys
import transformers
import torch
import time
import argparse
from tqdm import tqdm
import string
from RoBERTa import *
from LSTM import *
from GPT2 import *
from transformers import RobertaTokenizer
from transformers import AutoTokenizer
import logging    # first of all import the module
from datetime import datetime

import pandas as pd

parser = argparse.ArgumentParser("main")

parser.add_argument('--train_num_points', type=int,             default = 1000,            help='train data number')
parser.add_argument('--valid_num_points', type=int,             default = 500,               help='validation data number')
parser.add_argument('--report_num_points',type=int,             default = 500,              help='report number')
parser.add_argument('--model_name',       type=str,             default = 'roberta-scratch',   help='model name')
parser.add_argument('--max_length',       type=int,             default=32,                 help='max_length')
parser.add_argument('--num_labels',       type=int,             default=3,                 help='num_labels')
parser.add_argument('--batch_size',       type=int,             default=4,                help='Batch size')
parser.add_argument('--num_workers',      type=int,             default=0,                  help='num_workers')
parser.add_argument('--replace_size',     type=int,             default=3,                  help='to test sensitivity, we need to replance each word by x random words from vocab, here we specify the x')
parser.add_argument('--epochs',           type=int,             default=500,                  help='num of epochs')
parser.add_argument('--lr',               type=float,           default=1e-4,              help='lr')
parser.add_argument('--weight_decay',     type=float,           default=0.0,                 help='weight_decay')
parser.add_argument('--gamma',            type=float,           default=1,                 help='lr*gamma after each test')
parser.add_argument('--vocab_size',       type=int,             default=-1,                help='size of vocabulary')
parser.add_argument('--embedding_dim',    type=int,             default=256,               help='embedding dimension')
parser.add_argument('--hidden_dim',       type=int,             default=256,               help='hidden dimension')
parser.add_argument('--num_head',         type=int,             default=4,               help='num of head in lstm')
parser.add_argument('--num_layers',       type=int,             default=8,                 help='number of layers for LSTM and roberta')
parser.add_argument('--dropout',          type=float,             default=0.2,               help='dropout')
parser.add_argument('--pad_idx',          type=int,             default=1,                 help='ignores token with this index')
parser.add_argument('--sensitivity_method',type=str,            default='word',                 help='embedding/word')
parser.add_argument('--embedding_noise_variance',         type=float,            default=15,                 help='the variance of the noise for the embedding sensitivity testing')
parser.add_argument('--exp_name',                          type=str,            default='default',                 help='why you run this experiment?')
parser.add_argument('--dataset',                          type=str,            default='imdb',                 help='data name')
parser.add_argument('--roberta_act',        type=str, default='softmax' , help='softmax/relu')
parser.add_argument('--optimizer',        type=str, default='AdamW' , help='AdamW or SGD')
parser.add_argument('--cudaname',        type=str, default='cuda:9' , help='whichgpu')
args = parser.parse_args([])#(args=['--batch_size', '8',  '--no_cuda'])#used in ipynb

device = args.cudaname
#foldername = datetime.now().strftime(f'./logs/{args.dataset}/{args.model_name}/%Y_%m_%d_%H_%M_%S')
if 'roberta' in args.model_name and args.roberta_act == 'relu':
    foldername = f'./logs/{args.dataset}/{args.model_name}_{args.roberta_act}/{args.exp_name}'
else:
    foldername = f'./logs/{args.dataset}/{args.model_name}/{args.exp_name}'
# Create the folder if it doesn't exist
if not os.path.exists(foldername):
    os.makedirs(foldername)
# Define the log filename inside the newly created folder
logfilename = os.path.join(foldername, 'logfile.log')

logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setLevel(logging.DEBUG)

logging.basicConfig(filename=logfilename, filemode='w', format='%(asctime)s %(levelname)s - %(funcName)s: %(message)s')
handle = "root"
logger = logging.getLogger(handle)
logger.info(f'args:{args}')


if(args.dataset == 'boolq'):
    dataset = load_dataset('boolq')
elif (args.dataset in ['mnli', 'qqp', 'rte', 'qnli', 'mrpc']):
    dataset = load_dataset('glue', args.dataset )
elif(args.dataset == 'imdb'):
    dataset = load_dataset('stanfordnlp/imdb')
    contrast_data_validation = pd.read_csv('contrast_data_set/test_original.tsv', sep='\t')
    contrast_data_validation['Sentiment'] = contrast_data_validation['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)
    contrast_origin_data = {
        'label': contrast_data_validation['Sentiment'].tolist(),
        'text': contrast_data_validation['Text'].tolist(),
        'dataset':'imdb'
    }
    contrast_data_contrast = pd.read_csv('contrast_data_set/test_contrast.tsv', sep='\t')
    contrast_data_contrast['Sentiment'] = contrast_data_contrast['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)
    contrast_contrast_data = {
        'label': contrast_data_contrast['Sentiment'].tolist(),
        'text': contrast_data_contrast['Text'].tolist(),
        'dataset':'imdb'
    }

logger.info('\n Property of dataset:')
logger.info(f'train set size: {len(dataset["train"])}')
# logger.info(f'validation_mismatched set size: {len(dataset["validation_matched"])}')
# logger.info(f'test_matched set size: {len(dataset["test_matched"])}')
# logger.info(f'test_mismatched set size: {len(dataset["test_mismatched"])}')
# %%
# %%

train_num_points = min(args.train_num_points, len(dataset['train']))
train = dataset['train'].shuffle(seed=42).select(range(train_num_points))

if(args.dataset in ['boolq', 'qqp', 'rte', 'qnli', 'mrpc']):
    valid_num_points = min(args.valid_num_points, len(dataset['validation']))
    valid = dataset['validation'][-valid_num_points:]
    valid['dataset'] = args.dataset
    replaced = replaced_data_binary(valid, args.replace_size) 
    replaced['dataset'] = args.dataset
elif (args.dataset == 'mnli'):
    valid_num_points = min(args.valid_num_points, len(dataset['validation_matched']))
    valid = dataset['validation_matched'][-args.valid_num_points:]
    valid['dataset'] = args.dataset
    replaced = replaced_data(valid, args.replace_size) 
    replaced['dataset'] = args.dataset
elif (args.dataset == 'imdb'): #we just test the accuracy on contrast set, valid dataset now is the contrast origin.
    valid_num_points = min(args.valid_num_points, len(dataset['test']))
    shuffled_test = dataset['test'].shuffle(seed=42)
    valid = shuffled_test[-args.valid_num_points:]#valid = contrast_origin_data
    valid['dataset'] = args.dataset
    replaced = replaced_data_binary(valid, args.replace_size) 
    replaced['dataset'] = args.dataset
    contrastset = contrast_contrast_data

# Shuffle the subset and then select the number of points for evaluation
embedding_sens_eval_data = train.shuffle(seed=42).select(range(valid_num_points))[:]
train = train[:]
train['dataset'] = args.dataset
embedding_sens_eval_data['dataset'] = args.dataset


if args.model_name=='roberta-scratch' or args.model_name=='gpt2-scratch':
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
elif args.model_name=='lstm':
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
else:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name)
args.vocab_size = tokenizer.vocab_size
#mnli
#The Multi-Genre Natural Language Inference Corpus is a crowdsourced collection of sentence pairs with textual entailment annotations. Given a premise sentence and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). The premise sentences are gathered from ten different sources, including transcribed speech, fiction, and government reports. The authors of the benchmark use the standard test set, for which they obtained private labels from the RTE authors, and evaluate on both the matched (in-domain) and mismatched (cross-domain) section. They also uses and recommend the SNLI corpus as 550k examples of auxiliary training data.



# %%






# %%





/home/tianyizhou/anaconda3/envs/math/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
replaced.keys()

dict_keys([None, 'text', 'sequence_length', 'dataset'])

In [10]:
# %%
# pdb.set_trace()
contrast_data = None
if(args.dataset in ['boolq', 'rte', 'qqp', 'qnli', 'mrpc','imdb']):
    train_data = get_Dataset_binary(train, tokenizer, args.model_name, max_length=args.max_length)
    train_dataloader = DataLoader(train_data, sampler= SequentialSampler(train_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    valid_data = get_Dataset_binary(valid, tokenizer, args.model_name, max_length=args.max_length)
    valid_dataloader = DataLoader(valid_data, sampler=SequentialSampler(valid_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    if args.dataset!='imdb':
        replaced_data = get_Replaced_Dataset_binary(replaced, tokenizer, args.model_name, max_length = args.max_length)
        replaced_dataloader = DataLoader(replaced_data, sampler=SequentialSampler(replaced_data), 
                                batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    else:
        replaced_dataloader = None
    embedding_sens_eval_data = get_Dataset_binary(embedding_sens_eval_data, tokenizer, args.model_name, max_length = args.max_length)
    embedding_sens_eval_dataloader = DataLoader(embedding_sens_eval_data, sampler=SequentialSampler(replaced_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    print("Length of Embedding Sensitivity Eval Loader", len(embedding_sens_eval_dataloader))
elif (args.dataset == 'mnli'):
    train_data = get_Dataset(train, tokenizer, args.model_name, max_length=args.max_length)
    train_dataloader = DataLoader(train_data, sampler= SequentialSampler(train_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    valid_data = get_Dataset(valid, tokenizer, args.model_name, max_length=args.max_length)
    valid_dataloader = DataLoader(valid_data, sampler=SequentialSampler(valid_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    replaced_data = get_Replaced_Dataset(replaced, tokenizer, args.model_name, max_length = args.max_length)
    replaced_dataloader = DataLoader(replaced_data, sampler=SequentialSampler(replaced_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
    embedding_sens_eval_data = get_Dataset(embedding_sens_eval_data, tokenizer, args.model_name, max_length = args.max_length)
    embedding_sens_eval_dataloader = DataLoader(embedding_sens_eval_data, sampler=SequentialSampler(replaced_data), 
                            batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)
if args.dataset == 'imdb':                            
    contrast_data = get_Dataset_binary(contrastset, tokenizer, args.model_name, max_length=args.max_length)
    contrast_dataloader = DataLoader(contrast_data, sampler=SequentialSampler(contrast_data), 
                                batch_size=args.batch_size, pin_memory=args.num_workers>0, num_workers=args.num_workers)

# %%
if args.model_name=='roberta-scratch':
    model = TextClassifier(args,foldername).to(device)
elif args.model_name=='roberta-base':
    model = TextClassifier(args,foldername).to(device)
elif args.model_name=='gpt2-scratch':
    model = GPT2TextClassifier(args,foldername).to(device)
elif args.model_name=='lstm':
    model = LSTMTextClassifier(args,foldername).to(device)
model.train(train_dataloader,valid_dataloader,embedding_sens_eval_dataloader,replaced_dataloader,device,contrast_dataloader=contrast_dataloader)

sizeinmb = get_model_size(model)
logger.info(f'model size: {sizeinmb}MB')



IndentationError: expected an indented block (490470676.py, line 16)

In [2]:
contrast_data[0]


(tensor([    0,   250, 32007, 49069,  3809,  1589, 49007,  3809, 48709,  2264,
           130,  1617,    64,  6210,    47,    10, 14353,   822,   116,    83,
          1563,  9850,     4,  1941, 34329,  2444, 10698,   167,  1617,  6683,
             4,     2]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]),
 tensor(1))

In [3]:
print(contrast_data[0][2])
tokenizer.decode(contrast_data[0][0])


tensor(1)


'<s>A spoiler.<br /><br />What three words can guarantee you a terrific film? A Canadian Production. THE BRAIN fits those words perfectly.</s>'

In [4]:
print(train_data[-50:][2])
tokenizer.decode(train_data[17][0])


tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1])


"<s>Back in 1985 I caught this thing (I can't even call it a movie) on cable. I was in college and I was with a high</s>"

In [5]:
dataset['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be